In [27]:
from typing_extensions import TypedDict
from datetime import time

class AgentState(TypedDict):
    graph_info:str

## the graph_info is shared between every nodes

## Nodes are just python function
The first positional argument is the state, as defined above 
Because the state is TypedDict with schema as defined above, each node can access the key, graph_state, with state['graph_state]

each nodes returns a new value of the state key --> graph_state

by default the value returned by each node will override the prior state value


In [28]:
def start_playing(state: AgentState):
    print("started_playing")
    return {"graph_information":state["graph_info"]}

def cricket(state: AgentState):
    print("My cricket node has been called")
    return {"graph_information": state["graph_info"]+ "Cricket"}

def badminton(state: AgentState):
    print("My badminton node has been called")
    return {"graph_information": state["graph_info"]+ "Badminton"}



In [29]:
import random 
from typing import Literal

def random_play(state: AgentState) -> Literal['cricket', 'badminton']:
    graph_info = state['graph_info']

    if random.random() > 0.5:
        return "cricket"
    
    else:
        return "badminton"

## Graph Construction

Now we build the graph from the components defined above 

The StateGraph class is the graph class that we can use

First we initialise a StateGraph with the AgentState class defined above

Then we add our nodes and edges

We use the START node, a special node that sends user input to the graph, to indicate where to start our graph

The END node is a special node that represents a terminal node

Finally, we compile our graph to perform a few basic checks on the graph structure


In [34]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.graph import MermaidDrawMethod

graph = StateGraph(AgentState)

##n node_name and function_name
graph.add_node("start_play", start_playing)
graph.add_node("cricket", cricket)
graph.add_node("badminton", badminton)



## Schedule the flow of the graph

graph.add_edge(START, "start_play")
graph.add_conditional_edges("start_play", random_play)
graph.add_edge("cricket", END)
graph.add_edge("badminton", END)


## compile the graph
graph_builder = graph.compile()


print(graph_builder.get_graph().draw_mermaid())







---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	start_play(start_play)
	cricket(cricket)
	badminton(badminton)
	__end__([<p>__end__</p>]):::last
	__start__ --> start_play;
	start_play -.-> badminton;
	start_play -.-> cricket;
	badminton --> __end__;
	cricket --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [35]:
x = AgentState({'graph_info': "Taran"})
start_playing(x)

started_playing


{'graph_information': 'Taran'}

## Graph Invocation

In [50]:
graph_builder.invoke({"graph_info":"My name is Taran"})

started_playing
My badminton node has been called


{'graph_info': 'My name is Taran'}